## Preparing Dataframe

This is the description for process of transformation data scraped from Discogs website. Whole script is stored in module `lib.convert` as function `prepare_df( temp )`. It automatically run after successfull pagination and it transform temporary file into proper database, ready for further analysis.

<a href='scraping_engine.ipynb'> Scraping engine </a> already sorted out good quality records, now is time to clean data and some labels about possible profits. This will be done with `pandas`

In [1]:
import pandas as pd

temp = 'Electronic_PL_sample_database.csv'
def prepare_df( temp ):
    df = pd.read_csv(temp, encoding='utf-8')
    ...
    return df

df = prepare_df(temp)

Above is a just sample creation for this notebook. Next steps are introduced inside `prepare_df` function

### Basic data cleaning

In [2]:
df.id.count()

3492

In [3]:
df.any().isnull().head(5)

Unnamed: 0    False
id            False
title         False
price         False
ratio         False
dtype: bool

It is just routine check. Actually, it should not be possible to have `naN` value as scraping engine from `dprice.py` ensures to set `0` every time it has problem with retrieving data. This is separately comunicated on screen for further investigation (to do - put it into log).

The engine ensures correct data type descriptions, so there is nothing to do in here too.

In [4]:
df.dtypes.head(6)

Unnamed: 0      int64
id            float64
title          object
price         float64
ratio         float64
start         float64
dtype: object


For sure, we don't want to see records which have poor sale history. These would be one-time sales which can be easily recognized as they have the same value in `df['low']` and `df['high']`.

In [5]:
df['hi-low'] = df['high'] - df['low']

With `df.hi-low` prepared we can exclude from our Dataframe one-time sales, next to records that were `never sold` and some errors in conection with `price = 0`

In [6]:
df[df['last_sold'] != 'never_sold']
df = df[df['hi-low'] != 0]
df = df[df['price'] != 0]

It should not be a suprise if no rows have been deleted. `dprice.py` engine is related to 'wants' so it exclude such record in very beginning. It is common that records with <a href='scraping_engine.ipynb' id='wants'>wants</a> over 100 has already sale history with more than 2 sales. 

In [7]:
df.id.count()

3492

### Add more labels

<a id='bargain_label'></a>
Next add a `'bargain_label'` which reflects ratio parameter - count as relation between price and median.
We will start with list of tuples

In [8]:
ratio_with_label = [
    (0, 0.01, 'error'), (0.01, 0.85, 'very nice'), (0.85, 1.15, 'good'),
    (1.15, 1.8, 'expensive'), (1.8, 2.9, 'overpriced'), (3, 1000, 'greedy')
]

Each tuple refer to ratio bounds and label, so `(0.01, 0.85, 'very nice')` means all prices with ratio <b>less</b> than `0.85` and <b>more</b> than `0.01` will get label `very nice`. Next, we assign labels to new column `bargain_label` running simple 'for' loop

In [9]:
for i in ratio_with_label:
        df.loc[(df['ratio'] > i[0]) & (df['ratio'] <= i[1]), 'bargain_label'] = i[2]

Let's check and count assigned labels

In [10]:
df.bargain_label.value_counts()

expensive     1294
good          1005
overpriced     586
very nice      392
greedy         215
Name: bargain_label, dtype: int64

Good, now it is time to create another important label which will hold discount value. It will be held in column called `discount` and then used for final sorting.

In [11]:
df['discount'] = df['med'] - df['price']
df['discount'].sort_values(ascending=False)

494      18.40
1762     11.58
659      11.37
1973     11.10
1714     11.05
         ...  
2500   -330.00
1769   -373.75
3048   -374.03
1999   -476.46
557    -476.50
Name: discount, Length: 3492, dtype: float64

Positive values means benefit for buyer while negative is a benefit for seller (in relation to median price)

## Creating Database

Finally we can save our work with a function which takes already prepared Dataframe with destinated file path and create csv database. 

In [12]:
def create_db( df, location ):
    try:
        df.to_csv(location, index=True)
        print('File was successfully created -- {}'.format(location))
    except ValueError:
        print("Empty file, not created -- {}{}".format(location), ValueError)
        pass

Such file is ready for <a href='show.ipynb'> <b> analysis </b> </a> with `pandas`, `numpy` and `matplotlib`